DyslexiaLens

Text Ingestion

In [63]:
#text = input(Text Input: )
text = """Urban environments are dynamic and multifaceted, shaped by the continuous interaction of diverse populations, intricate economic activities, and expansive infrastructural networks. As cities experience growth and transformation, municipal authorities must navigate the complexities of rapid development while striving to ensure the equitable allocation of resources and opportunities for all residents. Effective urban planning demands thoughtful consideration of issues such as affordable housing, the establishment of sustainable transportation systems, public health initiatives, and the stewardship of natural resources. These endeavors require the integration of data-driven decision-making, meaningful stakeholder engagement, and the adoption of adaptive policy frameworks to guarantee that vulnerable and marginalized communities are not excluded from the advantages of urban expansion.
  
Despite ambitious objectives and well-intentioned strategies, the implementation of inclusive urban development projects is frequently impeded by bureaucratic inefficiencies, limited financial resources, and competing political interests. Such challenges often result in delays, fragmented services, and a lack of transparency, ultimately undermining the intended impact of these initiatives. Consequently, disadvantaged groups may continue to face barriers in accessing essential services, endure substandard living conditions, and experience persistent socio-economic disparities. This ongoing reality exists in stark contrast to the professed commitment of city leaders to fostering urban spaces that are both resilient and just, highlighting the need for ongoing evaluation, innovation, and collaboration in urban governance."""


CV(OCR extraction)

In [64]:
import pytesseract
from PIL import Image
import pdfplumber
import io

For PDF

In [65]:
def extract_text_from_pdf(pdf_bytes):
    text = ""
    with pdfplumber.open(io.BytesIO(pdf_bytes)) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text.strip()

In [66]:
with open("earth.pdf", "rb") as f:
    pdf_text = extract_text_from_pdf(f.read())
    print("Extracted from PDF:\n", pdf_text)

Extracted from PDF:
 Earth is the third planet from the Sun and the only astronomical object known to harbor life. About
71% of Earth's surface is covered with water, mostly by oceans. The remaining 29% consists of
continents and islands. Earth's atmosphere is rich in nitrogen and oxygen, protecting life by
blocking harmful solar radiation and reducing temperature extremes between day and night.
Earth has a layered structure with a solid inner core, a liquid outer core, a mantle, and a crust. The
outer layer, the crust, is broken into tectonic plates that move slowly over time. These movements
cause earthquakes, volcanic activity, and the formation of mountains. Earth also supports diverse
ecosystems, ranging from deserts to rainforests and polar regions. Human activities, however, have
started to impact Earth's climate and biodiversity, making environmental conservation an important
challenge for the future.


For Image

In [67]:
def extract_text_from_image(image_bytes):
    image = Image.open(io.BytesIO(image_bytes))
    text = pytesseract.image_to_string(image)
    return text.strip()

In [68]:
with open("image.png", "rb") as f:
    img_text = extract_text_from_image(f.read())
    print("Extracted from Image:\n", img_text)

Extracted from Image:
 Earth is the third planet from the Sun and the only astronomical object known to harbor life. About
71% of Earth's surface is covered with water, mostly by oceans. The remaining 29% consists of
continents and islands. Earth's atmosphere is rich in nitrogen and oxygen, protecting life by
blocking harmful solar radiation and reducing temperature extremes between day and night.

Earth has a layered structure with a solid inner core, a liquid outer core, a mantle, and a crust. The
outer layer, the crust, is broken into tectonic plates that move slowly over time. These movements
cause earthquakes, volcanic activity, and the formation of mountains. Earth also supports diverse
ecosystems, ranging from deserts to rainforests and polar regions. Human activities, however, have
started to impact Earth's climate and biodiversity, making environmental conservation an important
challenge for the future.


Preprocessing

1. Text Cleaning

In [69]:
import ftfy
import re

In [70]:
def clean_text(text):

    #to clear the encoding issues
    text = ftfy.fix_text(text)

    #to remove the non printable letters
    text = ''.join(c for c in text if c.isprintable())
    
    #to remove the extra spaces
    text = re.sub(r'[\r\n]+', '\n', text)
    text = re.sub(r'\s{2,}', ' ', text)

    return text.strip()

In [71]:
cleanText = clean_text(text)

2. Segmentation

In [72]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to
[nltk_data]     /home/prathoseraaj/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/prathoseraaj/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [73]:
def segmentation_text(text):
    paragraph = [p for p in text.split('\n') if p.strip()]
    sentance = []

    for para in paragraph:
        sentance.extend(nltk.sent_tokenize(para))

    tokens = [nltk.word_tokenize(sent) for sent in sentance]
    return {
        'paragraph': paragraph,
        'sentences': sentance,
        'tokens': tokens,
    }

In [74]:
preprocessed_text = segmentation_text(text)

Readability Assesment

In [75]:
full_text = " ".join(preprocessed_text["paragraph"])
sentences = preprocessed_text["sentences"]

print(full_text)
print(sentences)

Urban environments are dynamic and multifaceted, shaped by the continuous interaction of diverse populations, intricate economic activities, and expansive infrastructural networks. As cities experience growth and transformation, municipal authorities must navigate the complexities of rapid development while striving to ensure the equitable allocation of resources and opportunities for all residents. Effective urban planning demands thoughtful consideration of issues such as affordable housing, the establishment of sustainable transportation systems, public health initiatives, and the stewardship of natural resources. These endeavors require the integration of data-driven decision-making, meaningful stakeholder engagement, and the adoption of adaptive policy frameworks to guarantee that vulnerable and marginalized communities are not excluded from the advantages of urban expansion. Despite ambitious objectives and well-intentioned strategies, the implementation of inclusive urban develo

In [76]:
import spacy

In [77]:
nlp = spacy.load("en_core_web_sm")

In [78]:
import chromadb
from chromadb.utils import embedding_functions

chroma_client = chromadb.Client()

default_ef = embedding_functions.DefaultEmbeddingFunction()

try:
    chroma_client.delete_collection(name="dyslexia_guidelines")
except:
    pass

collection = chroma_client.create_collection(
    name="dyslexia_guidelines", 
    embedding_function=default_ef
)

expert_rules = [
    "Use active voice. Passive voice reverses the actor and action, increasing cognitive load.",
    "Split long sentences. Dyslexic working memory struggles with sentences over 15-20 words.",
    "Avoid double negatives. Constructs like 'not unlikely' cause processing delays.",
    "Clarify ambiguous pronouns. Do not rely on 'it' or 'they' if the reference is far back.",
    "Simplify polysyllabic vocabulary. Replace Latinate words (e.g., 'subsequently') with Anglo-Saxon equivalents.",
    "Use linear narrative flow. Dyslexic readers rely on chronological order.",
    "Explain abstract concepts with concrete imagery. Use visual metaphors."
]

ids = [f"rule_{i}" for i in range(len(expert_rules))]

collection.add(documents=expert_rules, ids=ids)
print("Knowledge Base Created with Expert Rules")

Knowledge Base Created with Expert Rules


1. Readability Score

In [79]:
import textstat

In [80]:
def readability_score(text):
    return{
        "flesch_reading_ease" : textstat.flesch_reading_ease(text),
        "flesch_kincaid_grade" : textstat.flesch_kincaid_grade(text),
        "gunning_fog": textstat.gunning_fog(text),
        "smog_index": textstat.smog_index(text),
        "coleman_liau_index" : textstat.coleman_liau_index(text),
        "automated_readability_index" : textstat.automated_readability_index(text),
        "dale_chall_readability_score" : textstat.dale_chall_readability_score(text),
        "difficult_words_count" : textstat.difficult_words(text),
        "difficult_words_list" : textstat.difficult_words_list(text),
        }

In [81]:
readability_score(text)

{'flesch_reading_ease': -19.15475290697674,
 'flesch_kincaid_grade': 22.60752906976745,
 'gunning_fog': 28.79651162790698,
 'smog_index': 23.02143706103936,
 'coleman_liau_index': 22.916279069767445,
 'automated_readability_index': 25.06512790697675,
 'dale_chall_readability_score': 14.884151162790697,
 'difficult_words_count': 119,
 'difficult_words_list': ['ultimately',
  'establishment',
  'environments',
  'guarantee',
  'frequently',
  'stewardship',
  'Consequently',
  'expansion',
  'conditions',
  'challenges',
  'wellintentioned',
  'natural',
  'urban',
  'adoption',
  'ambitious',
  'substandard',
  'adaptive',
  'decisionmaking',
  'highlighting',
  'initiatives',
  'transparency',
  'multifaceted',
  'ongoing',
  'expansive',
  'intricate',
  'consideration',
  'Effective',
  'activities',
  'fragmented',
  'engagement',
  'resilient',
  'impact',
  'issues',
  'continuous',
  'thoughtful',
  'strategies',
  'objectives',
  'financial',
  'diverse',
  'systems',
  'accessi

2. Detect long sentance

In [82]:
def detect_long_sentance(sentances, threshold=25):
    return [sent for sent in sentances if len(sent.split()) > threshold]

In [83]:
detect_long_sentance(sentances=sentences)

['As cities experience growth and transformation, municipal authorities must navigate the complexities of rapid development while striving to ensure the equitable allocation of resources and opportunities for all residents.',
 'Effective urban planning demands thoughtful consideration of issues such as affordable housing, the establishment of sustainable transportation systems, public health initiatives, and the stewardship of natural resources.',
 'These endeavors require the integration of data-driven decision-making, meaningful stakeholder engagement, and the adoption of adaptive policy frameworks to guarantee that vulnerable and marginalized communities are not excluded from the advantages of urban expansion.',
 'Despite ambitious objectives and well-intentioned strategies, the implementation of inclusive urban development projects is frequently impeded by bureaucratic inefficiencies, limited financial resources, and competing political interests.',
 'This ongoing reality exists in

3. Detect passive voice

In [84]:
import spacy

In [85]:
def detect_passive_voice(sentences):
    passive_sentences = []

    for sent in sentences:
        doc = nlp(sent)
        for token in doc:
            if token.dep_ == "nsubjpass":
                passive_sentences.append(sent)
                break

    return passive_sentences

In [86]:
detect_long_sentance(sentances=sentences)

['As cities experience growth and transformation, municipal authorities must navigate the complexities of rapid development while striving to ensure the equitable allocation of resources and opportunities for all residents.',
 'Effective urban planning demands thoughtful consideration of issues such as affordable housing, the establishment of sustainable transportation systems, public health initiatives, and the stewardship of natural resources.',
 'These endeavors require the integration of data-driven decision-making, meaningful stakeholder engagement, and the adoption of adaptive policy frameworks to guarantee that vulnerable and marginalized communities are not excluded from the advantages of urban expansion.',
 'Despite ambitious objectives and well-intentioned strategies, the implementation of inclusive urban development projects is frequently impeded by bureaucratic inefficiencies, limited financial resources, and competing political interests.',
 'This ongoing reality exists in

4. Detect ambigious structures

In [87]:
def detect_ambiguous_structures(sentences):
    ambiguous_keywords = ["might", "could", "possibly", "maybe", "potentially", "approximately", "suggests", "appears"]
    return [sent for sent in sentences if any(word in sent.lower() for word in ambiguous_keywords)]

In [88]:
detect_ambiguous_structures(sentences=sentences)

[]

5. Return the readability assesment 

In [89]:
def assesment_data(preprocessed_text):

    paragraph = "".join(preprocessed_text['paragraph'])
    sentences = preprocessed_text["sentences"]

    return{
        "readability_score" : readability_score(paragraph),
        "long_sentences" : detect_long_sentance(sentences),
        "passive_voice" : detect_passive_voice(sentences),
        "detect_ambiguous_structures" : detect_ambiguous_structures(sentences),
    }

In [90]:
results = assesment_data(preprocessed_text)

In [91]:
def retrieve_expert_advice(analysis_metrics):
    """
    Decides which rules to send to Gemini based on text analysis.
    """
    query_text = "general simplification"
    found_issues = []
    
    if len(analysis_metrics.get('passive_voice', [])) > 0:
        query_text += " fix passive voice"
        found_issues.append("Passive Voice")
        
    if len(analysis_metrics.get('long_sentences', [])) > 0:
        query_text += " split long sentences"
        found_issues.append("Long Sentences")
        
    if analysis_metrics['readability_score']['flesch_kincaid_grade'] > 12:
        query_text += " simplify complex vocabulary"
        found_issues.append("Complex Vocabulary")

    print(f"🔍 Diagnosis: Detected {found_issues}")

    results = collection.query(
        query_texts=[query_text],
        n_results=2 
    )
    
    return results['documents'][0]

Text Simplification

1. Universal replacement

In [92]:
UNIVERSAL_REPLACEMENTS = {
    "subsequently": "later",
    "nevertheless": "however", 
    "furthermore": "also",
    "demonstrate": "show",
    "facilitate": "help",
    "utilize": "use",
    "implement": "put into practice",
    "establish": "set up",
    "maintain": "keep",
    "ensure": "make sure",
    "require": "need",
    "provide": "give",
    "determine": "find out",
    "indicate": "show",
    "consider": "think about",
    "obtain": "get",
    "receive": "get",
    "purchase": "buy",
    "commence": "start",
    "complete": "finish",
    "continue": "keep going",
    "include": "have",
    "exclude": "leave out",
    "increase": "make bigger",
    "decrease": "make smaller",
    "improve": "make better",
    "reduce": "make less",
    "eliminate": "remove",
    "prevent": "stop",
    "achieve": "reach",
    "assist": "help",
    "contribute": "add to",
    "participate": "take part",
    "collaborate": "work together",
    "communicate": "talk",
    "recommend": "suggest",
    "advise": "tell",
    "inform": "let know",
    "notify": "tell",
    "request": "ask for",
    "submit": "send in",
    "approve": "say yes to",
    "reject": "say no to",
    "accept": "take",
    "acknowledge": "recognize",
    "confirm": "make sure",
    "verify": "check",
    "examine": "look at",
    "evaluate": "judge",
    "assess": "check",
    "analyze": "study",
    "investigate": "look into",
    "research": "study",
    "monitor": "watch",
    "supervise": "oversee",
    "manage": "run",
    "operate": "run",
    "function": "work",
    "perform": "do",
    "execute": "carry out",
    "conduct": "do",
    "organize": "arrange",
    "coordinate": "work together",
    "schedule": "plan time",
    "allocate": "give out",
    "distribute": "share out",
    "deliver": "bring",
    "transport": "move",
    "transfer": "move",
    "relocate": "move",
    "install": "put in",
    "construct": "build",
    "manufacture": "make",
    "produce": "make",
    "generate": "create",
    "develop": "make",
    "design": "plan",
    "modify": "change",
    "adjust": "change",
    "adapt": "change to fit",
    "customize": "make to fit",
    "enhance": "improve",
    "upgrade": "improve",
    "update": "make current",
    "revise": "change",
    "edit": "change",
    "correct": "fix",
    "repair": "fix",
    "resolve": "solve",
    "address": "deal with",
    "handle": "deal with",
    "process": "deal with",
    "procedure": "way to do",
    "protocol": "rules",
    "policy": "rule",
    "regulation": "rule",
    "requirement": "need",
    "specification": "details",
    "criteria": "standards",
    "standard": "normal way",
    "guideline": "guide",
    "instruction": "directions",
    "documentation": "papers",
    "information": "facts",
    "data": "facts",
    "evidence": "proof",
    "reference": "source",
    "resource": "tool",
    "material": "stuff",
    "equipment": "tools",
    "instrument": "tool",
    "device": "machine",
    "system": "way of doing",
    "method": "way",
    "technique": "way",
    "approach": "way",
    "strategy": "plan",
    "solution": "answer",
    "alternative": "other choice",
    "option": "choice",
    "opportunity": "chance",
    "possibility": "chance",
    "potential": "possible",
    "capacity": "ability",
    "capability": "ability",
    "expertise": "skill",
    "knowledge": "knowing",
    "experience": "practice",
    "qualification": "training",
    "credential": "proof of training",
    "certificate": "proof",
    "license": "permission",
    "permit": "permission",
    "authorization": "permission",
    "approval": "saying yes",
    "consent": "saying yes",
    "agreement": "deal",
    "contract": "deal",
    "arrangement": "plan",
    "commitment": "promise",
    "obligation": "duty",
    "responsibility": "job",
    "accountability": "being responsible",
    "liability": "being responsible for",
    "consequence": "result",
    "outcome": "result",
    "effect": "result",
    "impact": "effect",
    "influence": "effect on",
    "benefit": "good thing",
    "advantage": "good point",
    "disadvantage": "bad point",
    "limitation": "limit",
    "restriction": "limit",
    "constraint": "limit",
    "barrier": "block",
    "obstacle": "block",
    "challenge": "hard thing",
    "difficulty": "hard thing",
    "problem": "trouble",
    "issue": "problem",
    "concern": "worry",
    "risk": "danger",
    "hazard": "danger",
    "safety": "being safe",
    "security": "being safe",
    "protection": "keeping safe",
    "prevention": "stopping",
    "emergency": "urgent problem",
    "urgent": "needs doing now",
    "immediate": "right now",
    "priority": "most important",
    "significant": "important",
    "essential": "needed",
    "necessary": "needed",
    "required": "needed",
    "mandatory": "must do",
    "optional": "choice",
    "voluntary": "choice",
    "automatic": "by itself",
    "manual": "by hand",
    "individual": "single",
    "personal": "your own",
    "private": "not public",
    "public": "for everyone",
    "general": "for most",
    "specific": "exact",
    "particular": "special",
    "unique": "one of a kind",
    "common": "normal",
    "typical": "normal",
    "standard": "normal",
    "regular": "normal",
    "frequent": "often",
    "occasional": "sometimes",
    "rare": "not often",
    "unusual": "not normal",
    "exceptional": "very special",
    "extraordinary": "amazing",
    "remarkable": "worth noting",
    "notable": "worth noting",
    "important": "matters",
    "valuable": "worth a lot",
    "useful": "helpful",
    "effective": "works well",
    "efficient": "works well with less",
    "successful": "works",
    "beneficial": "helpful",
    "positive": "good",
    "negative": "bad",
    "neutral": "neither good nor bad",
    "reasonable": "makes sense",
    "appropriate": "right for",
    "suitable": "right for",
    "adequate": "enough",
    "sufficient": "enough",
    "insufficient": "not enough",
    "excessive": "too much",
    "minimum": "least",
    "maximum": "most",
    "average": "normal amount",
    "approximately": "about",
    "exactly": "just right",
    "precisely": "exactly",
    "accurately": "correctly",
    "correctly": "right way",
    "properly": "right way",
    "effectively": "in a way that works",
    "efficiently": "without waste",
    "quickly": "fast",
    "slowly": "not fast",
    "immediately": "right now",
    "eventually": "in the end",
    "finally": "at last",
    "initially": "at first",
    "originally": "at first",
    "previously": "before",
    "currently": "now",
    "presently": "now",
    "recently": "not long ago",
    "temporarily": "for a short time",
    "permanently": "forever",
    "continuously": "without stopping",
    "frequently": "often",
    "regularly": "on schedule",
    "consistently": "always the same way",
    "constantly": "all the time"
}

2. lexical simplify

In [93]:
def lexical_simplify(text):
    """Replace complex words with simpler ones"""
    words = nltk.word_tokenize(text)
    simplified_words = []
    
    for word in words:
        # Clean the word for lookup (remove punctuation)
        clean_word = re.sub(r'[^\w]', '', word.lower())
        
        if clean_word in UNIVERSAL_REPLACEMENTS:
            replacement = UNIVERSAL_REPLACEMENTS[clean_word]
            # Preserve original capitalization
            if word and word[0].isupper():
                replacement = replacement.capitalize()
            # Keep punctuation if it was attached
            if word != clean_word and len(word) > len(clean_word):
                punct = word[len(clean_word):]
                replacement += punct
            simplified_words.append(replacement)
        else:
            simplified_words.append(word)
    
    return ' '.join(simplified_words)

3. Sentance split

In [94]:
def sentence_split(sentence, max_words=18):
    """Split long sentences at natural break points"""
    words = sentence.split()
    if len(words) <= max_words:
        return [sentence]
    
    # Look for natural connectors to split on
    connectors = [' and ', ' but ', ' however ', ' although ', ' while ', ' because ']
    
    for connector in connectors:
        if connector in sentence:
            parts = sentence.split(connector, 1)
            if len(parts) == 2 and len(parts[0].split()) > 8:
                first_part = parts[0].strip()
                second_part = parts[1].strip()
                
                # Add proper punctuation
                if not first_part.endswith('.'):
                    first_part += '.'
                if not second_part[0].isupper():
                    second_part = second_part.capitalize()
                if not second_part.endswith('.'):
                    second_part += '.'
                
                return [first_part, second_part]
    
    # Fallback: split at comma
    if ',' in sentence:
        comma_pos = sentence.rfind(',', 0, len(sentence)//2 + 30)
        if comma_pos > len(sentence)//4:
            first_part = sentence[:comma_pos].strip() + '.'
            second_part = sentence[comma_pos+1:].strip()
            if not second_part[0].isupper():
                second_part = second_part.capitalize()
            if not second_part.endswith('.'):
                second_part += '.'
            return [first_part, second_part]
    
    # Last resort: split in middle
    mid = len(words) // 2
    first_part = ' '.join(words[:mid]) + '.'
    second_part = ' '.join(words[mid:])
    if not second_part[0].isupper():
        second_part = second_part.capitalize()
    if not second_part.endswith('.'):
        second_part += '.'
    
    return [first_part, second_part]


4. Fix grammar Issues

In [95]:
def fix_grammar_issues(sentence):
    """Fix common grammar problems after simplification"""
    
    # Fix awkward constructions
    sentence = sentence.replace("putting into practice of", "implementing")
    sentence = sentence.replace("the improvement of", "improving")
    sentence = sentence.replace("the distribution of", "distributing")
    sentence = sentence.replace("connecting different transport connections", "transport connections")
    sentence = sentence.replace("rule-based following rules", "following regulations")
    sentence = sentence.replace("giving power", "empowerment")
    
    # Fix spacing around punctuation
    sentence = re.sub(r'\s+,', ',', sentence)  # Remove space before comma
    sentence = re.sub(r',\s*\.', '.', sentence)  # Remove ",." combinations
    sentence = re.sub(r'\s+\.', '.', sentence)   # Remove space before period
    
    # Ensure proper sentence structure
    sentence = sentence.strip()
    if sentence:
        sentence = sentence[0].upper() + sentence[1:]
        if not sentence.endswith('.'):
            sentence += '.'
    
    return sentence

5. Merge sentance fragments

In [96]:
def merge_sentence_fragments(sentences):
    """Combine sentence fragments that start with 'And' or 'But'"""
    merged = []
    i = 0
    
    while i < len(sentences):
        current = sentences[i].strip()
        
        # Check if next sentence is a fragment
        if i + 1 < len(sentences):
            next_sent = sentences[i + 1].strip()
            if next_sent.startswith(('And ', 'But ', 'Or ')):
                # Merge with previous sentence
                connector = next_sent.split()[0].lower()
                rest = ' '.join(next_sent.split()[1:])
                
                # Remove period from current sentence and add connector
                if current.endswith('.'):
                    current = current[:-1]
                merged_sentence = f"{current} {connector} {rest}"
                merged.append(merged_sentence)
                i += 2  # Skip both sentences
                continue
        
        merged.append(current)
        i += 1
    
    return merged

6. Apply all simplification Techniques

In [97]:
def comprehensive_simplify(text):    
    grade_level = textstat.flesch_kincaid_grade(text)
    
    if grade_level <= 12:
        max_sentence_length = 20
    elif grade_level <= 20:
        max_sentence_length = 18
    else:
        max_sentence_length = 15
    
    sentences = nltk.sent_tokenize(text)
    simplified_sentences = []
    
    for sentence in sentences:
        split_sentences = sentence_split(sentence, max_sentence_length)
        
        for split_sent in split_sentences:
            
            simplified = fix_grammar_issues(split_sent)
            simplified_sentences.append(simplified)
    
    simplified_sentences = merge_sentence_fragments(simplified_sentences)
    return simplified_sentences

7. Calculate similarity

In [98]:
def calculate_similarity(original, simplified_list):
    """Check how much meaning is preserved"""
    simplified_text = ' '.join(simplified_list)
    
    orig_words = set(original.lower().split())
    simp_words = set(simplified_text.lower().split())
        
    # Account for known replacements
    adjusted_simp = set()
    for word in simp_words:
        original_word = None
        for complex_word, simple_word in UNIVERSAL_REPLACEMENTS.items():
            if simple_word.lower() == word.lower():
                original_word = complex_word
                break
        
        if original_word and original_word in orig_words:
            adjusted_simp.add(original_word)
        else:
            adjusted_simp.add(word.lower())
    
    overlap = len(orig_words.intersection(adjusted_simp))
    total = len(orig_words.union(adjusted_simp))
    
    return overlap / total if total > 0 else 0

In [99]:
original_grade = textstat.flesch_kincaid_grade(text)
difficult_words = textstat.difficult_words(text)
simplified_sentences = comprehensive_simplify(text)
simplified_text = ' '.join(simplified_sentences)
final_grade = textstat.flesch_kincaid_grade(simplified_text)
improvement = original_grade - final_grade
similarity = calculate_similarity(text, simplified_sentences)

full_result = f"""
=== UNIVERSAL DYSLEXIALENS SIMPLIFICATION ===
Original text grade level: {original_grade:.1f}
Difficult words: {difficult_words}
Text complexity: Grade {original_grade:.1f} - maximum simplification

=== SIMPLIFIED TEXT ===
""" + '\n'.join([f"{i+1}. {s}" for i, s in enumerate(simplified_sentences)]) + f"""

=== RESULTS ===
Original grade level: {original_grade:.1f}
Simplified grade level: {final_grade:.1f}
Improvement: {improvement:.1f} grades easier
Meaning preservation: {similarity:.2f}
"""

print(full_result)


=== UNIVERSAL DYSLEXIALENS SIMPLIFICATION ===
Original text grade level: 22.6
Difficult words: 119
Text complexity: Grade 22.6 - maximum simplification

=== SIMPLIFIED TEXT ===
1. Urban environments are dynamic and multifaceted, shaped by the continuous interaction of diverse populations.
2. Intricate economic activities, and expansive infrastructural networks.
3. As cities experience growth and transformation, municipal authorities must navigate the complexities of rapid development.
4. Striving to ensure the equitable allocation of resources and opportunities for all residents.
5. Effective urban planning demands thoughtful consideration of issues such as affordable housing, the establishment of sustainable transportation systems, public health initiatives.
6. The stewardship of natural resources.
7. These endeavors require the integration of data-driven decision-making, meaningful stakeholder engagement.
8. The adoption of adaptive policy frameworks to guarantee that vulnerable and

LLM process

In [100]:
import google.generativeai as genai
import os
from dotenv import load_dotenv

/home/prathoseraaj/Documents/DyslexiaLens/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_16785/1674680142.py:1: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  import google.generativeai as genai


In [101]:
load_dotenv()
gemini_api_key = os.getenv("gemini_api_key")
genai.configure(api_key=gemini_api_key)

In [102]:
def gemini_process_rag(text, full_result, expert_guidelines, difficult_words_list):
    
    model = genai.GenerativeModel("gemini-2.5-flash")

    rules_text = "\n".join([f"- {rule}" for rule in expert_guidelines])
    
    hard_words = list(set(difficult_words_list))[:10]
    hard_words_str = ", ".join(hard_words)

    prompt = f"""
    You are an expert at creating dyslexia-friendly text.

    Here is the original text:
    {text}

    Here is the draft simplified version (for reference):
    {full_result}
    
    *** RAG RESEARCH CONTEXT (Mandatory) ***
    1. Difficult Vocabulary Detected: [{hard_words_str}] -> Replace these with simpler synonyms.
    2. Expert Guidelines to Follow:
    {rules_text}

    Instructions:
    - Use the original text as the primary source of truth.
    - DO NOT SUMMARIZE. Keep every single fact, detail, and nuance.
    - Simply rewrite the sentences to be shorter and clearer (Subject-Verb-Object).
    - Use the 'Draft Simplified Version' only to help with sentence structure, but ensure no meaning is lost.
    - The final result should be the EXACT same content, just easier to read.

    Return only the completed simplified text.
    """

    response = model.generate_content(prompt)
    return response.text

In [103]:
# 1. Run Analysis (You likely ran this earlier)
results = assesment_data(preprocessed_text) 

# 2. Retrieve Expert Rules based on that analysis
my_guidelines = retrieve_expert_advice(results)

# 3. Get the list of hard words
hard_words = results['readability_score']['difficult_words_list']

# 4. Run Gemini with RAG + Vocabulary Targeting
final_output = gemini_process_rag(text, full_result, my_guidelines, hard_words)

print("\n=== FINAL RAG OUTPUT ===\n")
print(final_output)

🔍 Diagnosis: Detected ['Passive Voice', 'Long Sentences', 'Complex Vocabulary']

=== FINAL RAG OUTPUT ===

Urban areas are complex places. They are always changing. Many different people live and interact there. Complex money-making activities happen. Large systems like roads and pipes are also part of them.

Cities grow and change quickly. City leaders must deal with this fast growth. They must work to share money, help, and chances fairly for everyone. Good city planning needs careful thought. It covers things like affordable homes. It includes setting up long-lasting travel methods. It also covers plans for people's health. And it means caring for nature.

These efforts need facts to make choices. They need to truly involve everyone affected. They also need flexible rules. This is to make sure groups that are easily hurt or left out get the benefits of city growth.

But plans for fair city growth often face problems. These plans might have good aims. Yet, they are often blocked by s